In [11]:
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

import matplotlib.pyplot as plt
import numpy as np
import cvxpy as cp
import time


In [24]:
def sw_ctrl_pers(A, b, x0, T, x_max, binary):
    n = len(x0)
    K = len(A)
    
    start_time = time.time()
    
    # decision variables
    z = cp.Variable((n * K, T))
    x = cp.Variable((n, T+1))
    
    if binary:
        s = cp.Variable((K, T), boolean=True)
    else:
        s = cp.Variable((K, T))

    # objective
    J = cp.sum_squares(x[:,T])
    for t in range(T-1):
        for i in range(K):
            J += cp.quad_over_lin(z[n*i:n*(i+1),t+1], s[i,t+1])
            
    objective = cp.Minimize(J)

    # dynamics
    constraints = []
    for t in range(T):
        Azb_sum = 0
        for i in range(K):
            Azb_sum += A[i] @ z[n*i:n*(i+1),t] + (b[i].flatten() * s[i,t])
        constraints.append(x[:,t+1] == Azb_sum)

    # other constraints
    constraints.append(0 <= s)
    constraints.append(s <= 1)
    constraints.append(cp.sum(s, axis=0) == 1)
    
    def sum_n_elements_z(z, K, n, T):
        z_reshaped = cp.reshape(z, (K, n*T))
        z_sum = cp.sum(z_reshaped, axis=0)
        z_sum_reshaped = cp.reshape(z_sum, (n, T))
        return z_sum_reshaped
    
    constraints.append(x[:,:T] == sum_n_elements_z(z, K, n, T))


    # state constraint
    for t in range(T-1):
        for i in range(K):
            constraints.append(cp.abs(z[n*i:n*(i+1),t+1]) <= s[i,t+1]*x_max)
    constraints.append(cp.abs(x[:,T]) <= x_max)

    # initial condition
    for i in range(K):
        constraints.append(z[n*i:n*(i+1),0] == (x0*s[i,0]).flatten())

    # problem
    prob = cp.Problem(objective, constraints)
    prob.solve()

    cpu_time = time.time() - start_time
    J_star = prob.value
    
    return J_star, x.value, s.value, cpu_time


# problem parameters
n = 2
K = 3
T = 10
x_max = 5
A = []
b = []
for i in range(K):
    A.append(np.eye(n) + 0.1 * np.random.randn(n, n))
    b.append(0.1 * np.random.randn(n, 1))
    x0 = np.random.randn(n, 1)


# PERSPECTIVE-BASED FORMULATION
# global solution
J_star_pers, x_pers, s_pers, runtime_pers = sw_ctrl_pers(A, b, x0, T, x_max, True)

# lower bound (integer-relaxation)
lb_pers, _, _, _ = sw_ctrl_pers(A, b, x0, T, x_max, False)

# upper bound (relax-and-round)
_, _, s, _ = sw_ctrl_pers(A, b, x0, T, x_max, False)
u = np.argmax(s, axis=0)
x_sim = x0
for t_sim in range(T):
    x_sim = np.concatenate((x_sim, A[u[t_sim]] @ x_sim[:, t_sim] + b[u[t_sim]]), axis=1)
ub_pers_rr = np.sum(x_sim[:, 1:] ** 2)
if any(np.linalg.norm(x_sim[:, 1:], np.inf, axis=0) > x_max):
    ub_pers_rr = float('inf')

# upper bound (shrinking-horizon heuristic)
x_sim = x0
for t_sim in range(T):
    _, _, s, _ = sw_ctrl_pers(A, b, x_sim[:, t_sim:t_sim+1], T - t_sim + 1, x_max, False)
    u = np.argmax(s[:, 0])
    x_sim = np.concatenate((x_sim, A[u] @ x_sim[:, t_sim] + b[u]), axis=1)
ub_pers = np.sum(x_sim[:, 1:] ** 2)
if any(np.linalg.norm(x_sim[:, 1:], np.inf, axis=0) > x_max):
    ub_pers = float('inf')

print('\n\n')
print(f'perspective relaxation lower bound is : {lb_pers:.3f}')
print(f'perspective shrinking-horizon upper bound is : {ub_pers:.3f}')
print(f'perspective relax-and-round upper bound is : {ub_pers_rr:.3f}')
print(f'perspective formulation (global) solution : {J_star_pers:.3f}')
print('\n')




perspective relaxation lower bound is : 0.134
perspective shrinking-horizon upper bound is : 2.994
perspective relax-and-round upper bound is : 3.207
perspective formulation (global) solution : nan




In [27]:
import cvxpy as cp

# Variables
x = cp.Variable()
s = cp.Variable(nonneg=True)  # 's' should be nonnegative

# Defining a function
f = cp.square(x)

# Defining the perspective function
perspective_f = cp.perspective(f, s)

# Defining the constraints
constraints = []  # Set any necessary constraints according to your problem

# Objective function (let's say we want to minimize the perspective of the function)
objective = cp.Minimize(perspective_f)

# Problem definition
problem = cp.Problem(objective, constraints)

# Solving the problem
problem.solve()

# Printing the optimal values
print("Optimal value for x: ", x.value)
print("Optimal value for s: ", s.value)


Optimal value for x:  -0.0
Optimal value for s:  0.433468995251828
